## Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
%matplotlib inline

## Downloading the Rice Image Dataset

Dataset Link - https://www.kaggle.com/datasets/muratkokludataset/rice-image-dataset/data

The Dataset is available in Kaggle. 


1.   Steps for downloading the dataset : 

*   !pip install kaggle
*   ! mkdir ~/.kaggle
*   ! cp kaggle.json ~/.kaggle/
*   ! chmod 600 ~/.kaggle/kaggle.json
*   ! kaggle datasets download muratkokludataset/rice-image-dataset

2.   Steps for unzipping the dataset :

      import zipfile <br>
      zip_file_path = '/content/rice-image-dataset.zip' <br>
      with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:<br>
      &emsp; zip_ref.extractall('/content/')

## Loading Dataset

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255, validation_split=0.3)
train_ds = train_datagen.flow_from_directory('Rice_Image_Dataset', subset = 'training', target_size=(256,256), batch_size = 32)
val_ds = train_datagen.flow_from_directory('Rice_Image_Dataset', subset = 'validation', target_size=(256,256), batch_size = 32)

Data augmentation is a technique used in machine learning and computer vision, particularly in the context of training deep learning models, to artificially increase the diversity of the training dataset. The idea is to apply various random transformations to the existing training data, creating new variations of the input samples. This helps the model generalize better to unseen data and improves its robustness. In the context of image data, data augmentation typically involves applying random transformations to images. Common augmentations include:


*   Rotation: Rotating the image by a random angle.
*   Width and Height Shifts: Shifting the image horizontally and vertically by a fraction of its total width and height, respectively.
*   Zooming: Zooming into or out of the image.
*   Flipping: Flipping the image horizontally or vertically.
*   Shearing: Shearing the image by a certain angle.
*   Brightness and Contrast Adjustments: Changing the brightness and contrast of the image.


By applying these random transformations, the model sees slightly different versions of the input data during each epoch of training. 
This helps the model become more invariant to variations in the input data and reduces overfitting. Data augmentation is particularly useful when the available training dataset is limited.

In the context of deep learning frameworks like TensorFlow and Keras, the ImageDataGenerator class is often used to perform data
augmentation. This class allows you to specify various augmentation parameters, and it generates augmented images on-the-fly during the 
training process.



In [ ]:
image_shape = train_ds.image_shape
num_classes = len(train_ds.class_indices)

print("Image shape in train dataset:", train_ds.image_shape)
print("Image shape in validation dataset:", val_ds.image_shape)
print("Number of classes:", num_classes)

## Visualisation

In [ ]:
batch_images, batch_labels = next(train_ds)
num_images = 16

fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(10, 10))
axes = axes.ravel()  

num_classes = batch_labels.shape[-1]  
rice_class_names = {0: 'Arborio', 1: 'Basmati', 2: 'Ipsala', 3: 'Jasmine', 4: 'Karacadag'}

for i in range(num_images):
    ax = axes[i]
    img = batch_images[i]
    one_hot_label = batch_labels[i]
    class_idx = np.argmax(one_hot_label) 
    class_name = rice_class_names.get(class_idx, 'Unknown')  
    ax.imshow(img)
    ax.set_title(f"Rice Type: {class_name}")
    ax.axis('off')

plt.tight_layout()
plt.show()

## Model Creation

In [ ]:
model = Sequential()

model.add(Conv2D(8, (4, 4), activation='relu', input_shape = image_shape, padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D((8,8), strides=(8,8)))
model.add(Dropout(0.25))

model.add(Conv2D(16, (4, 4), activation='relu', input_shape = image_shape, padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D((8,8), strides=(8,8)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
model.summary()

## Early Stopping

In [ ]:
early_stop = EarlyStopping(monitor = 'val_loss', patience = 2)

## Compile & Train Model

In [ ]:
model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics=['accuracy'], )
history = model.fit(train_ds, epochs = 10, validation_data = val_ds, callbacks = [early_stop])

We can see that the loss is around 2 and the accuracy is more than 98%,,,which is very good

For evaluating the model's performnace we can use Confusion Matrix, Classification Report, Visualize Training and Validation Accuracy/Loss

In multiclass classification problems, where there are more than two classes, there are two main strategies to transform the problem into multiple binary classification problems: One-vs-One (OvO) and One-vs-All (OvA), also known as One-vs-Rest (OvR).

**One-vs-One (OvO)**:
In the OvO approach, a separate binary classifier is trained for each pair of classes. If there are N classes, N(N-1)/2 binary classifiers are trained, each distinguishing between a pair of classes. During prediction, each binary classifier votes for one of the two classes, and the class with the most votes is selected as the final prediction.

For example, if there are three classes (A, B, and C), three binary classifiers would be trained:
1. A vs. B
2. A vs. C
3. B vs. C

To classify a new instance, all three binary classifiers make predictions, and the class with the most votes is chosen as the final prediction.

**One-vs-All (OvA) or One-vs-Rest (OvR)**:
In the OvA approach, N binary classifiers are trained, one for each class. For each class, a binary classifier is trained to distinguish that class from all the other classes combined. During prediction, the classifier with the highest confidence score or probability is selected, and the corresponding class is the predicted class.

For example, if there are three classes (A, B, and C), three binary classifiers would be trained:
1. A vs. (B+C)
2. B vs. (A+C)
3. C vs. (A+B)

To classify a new instance, all three binary classifiers make predictions, and the classifier with the highest confidence or probability determines the predicted class.

The choice between OvO and OvA depends on the specific problem and the characteristics of the dataset, such as the number of classes, the class distribution, and the complexity of the decision boundaries. In general, OvO tends to perform better when there are many classes, while OvA can be more efficient for problems with fewer classes.

But here as I used Deep Learning to solve the problem it is important to note that deep learning frameworks, such as TensorFlow and PyTorch, often handle multiclass classification problems directly, without explicitly implementing OvO or OvA strategies. Instead, they use techniques like softmax activation and cross-entropy loss to automatically handle multiple classes during training and prediction.